In [1]:
from agent import workflow
!pip install python-dotenv

D:\LangChain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Defaulting to user installation because normal site-packages is not writeable


In [2]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview",temperature=0)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [4]:
#create a state

class BlogState(TypedDict):
    title:str
    outline:str
    content:str


In [5]:
def create_outline(state:BlogState) -> BlogState:
    title = state["title"]
    outline = llm.invoke(f"Write an outline for a blog post about {title}")
    state["outline"] = outline
    return state



In [6]:
def create_blog(state:BlogState) -> BlogState:
    outline = state["outline"]
    title = state["title"]
    content = llm.invoke(f"Write a detailed blog for a this post whose title is {title} and outline is \n {outline}")
    state["content"] = content
    return state

In [7]:
graph = StateGraph(BlogState)
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)
graph.add_edge(START,"create_outline")
graph.add_edge("create_outline","create_blog")
graph.add_edge("create_blog",END)
workflow = graph.compile()

In [ ]:
initial_state = {'title':'Rise of AI in India'}
final_state = workflow.invoke(initial_state)
print(final_state)